In [1]:
import psycopg2
import pandas as pd

host = 'localhost'
user = 'postgres'
password = '1234'
database = 'postgres'

conn = psycopg2.connect(host=host, user=user, password=password, database=database)
cur = conn.cursor()

cols = ['Year_Birth', 'Education', 'Marital_Status', 'Income',
        'Kidhome', 'Teenhome', 'MntWines']
cols_str = ', '.join(cols)

cur.execute(f'SELECT {cols_str} FROM test')
datas = cur.fetchall()
conn.close()

df = pd.DataFrame(columns=cols, data=datas)
df

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,MntWines
0,1957,Graduation,Single,58138.0,0,0,635
1,1954,Graduation,Single,46344.0,1,1,11
2,1965,Graduation,Together,71613.0,0,0,426
3,1984,Graduation,Together,26646.0,1,0,11
4,1981,PhD,Married,58293.0,1,0,173
...,...,...,...,...,...,...,...
2235,1967,Graduation,Married,61223.0,0,1,709
2236,1946,PhD,Together,64014.0,2,1,406
2237,1981,Graduation,Divorced,56981.0,0,0,908
2238,1956,Master,Together,69245.0,0,1,428


In [2]:
df.isna().sum()

Year_Birth         0
Education          0
Marital_Status     0
Income            24
Kidhome            0
Teenhome           0
MntWines           0
dtype: int64

In [3]:
print(df.Education.unique())
print(df.Marital_Status.unique())

['Graduation' 'PhD' 'Master' 'Basic' '2n Cycle']
['Single' 'Together' 'Married' 'Divorced' 'Widow' 'Alone' 'Absurd' 'YOLO']


In [4]:
from sklearn.model_selection import train_test_split

features = ['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome']
target = 'MntWines'

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1792, 6), (448, 6), (1792,), (448,))

In [5]:
from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import pickle

pipe = make_pipeline(
    OneHotEncoder(),
    SimpleImputer(),
    StandardScaler(),
    LinearRegression(n_jobs=-1)
)
pipe.fit(X_train, y_train)

print('검증세트 정확도', pipe.score(X_test, y_test))

# some = pd.DataFrame([[1957, 'PhD', 'Married', 80000.0, 0, 0]], columns=features)
some = [{'Year_Birth': 1957, 'Education': 'PhD', 'Marital_Status': 'Married', 'Income': 80000.0, 'Kidhome': 0, 'Teenhome': 0}]
print(some)
print(pipe.predict(some)[0])

with open('ml_pipe.pkl','wb') as pickle_file:
    pickle.dump(pipe, pickle_file)


검증세트 정확도 0.44248204591431517
[{'Year_Birth': 1957, 'Education': 'PhD', 'Marital_Status': 'Married', 'Income': 80000.0, 'Kidhome': 0, 'Teenhome': 0}]
627.1972726049964


c:\Users\Eom\anaconda3\envs\test1\lib\site-packages\category_encoders\one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
c:\Users\Eom\anaconda3\envs\test1\lib\site-packages\category_encoders\one_hot.py:145: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for cat_name, class_ in values.iteritems():
